<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Deployment of a model to an Azure Container Instance (ACI)

## Table of contents <a id="table_of_content"></a>

1. [Introduction](#intro)
1. [Model retrieval and export](#model)
1. [Model deployment on Azure](#deploy)
  1. [Workspace retrieval](#workspace)
  1. [Model registration](#register)
  1. [Scoring script](#scoring)
  1. [Using AzureML Environments for Environment setup](#env)
  1. [Computational resources - Azure Container Instance](#compute)
  1. [Web service deployment](#websvc)
1. [Notes on web service deployment](#notes)
1. [Clean-up](#clean)
1. [Next steps](#next-steps)

## 1. Introduction <a id="intro"></a>

While building a good performing model is important, for it to be useful, it needs to be accessible. In this notebook, we will learn how to make this possible by deploying our model onto Azure. We will more particularly see how to:
- Register a model there
- Create a Docker image that contains our model
- Deploy a web service on [Azure Container Instances](https://azure.microsoft.com/en-us/services/container-instances/) using this Docker image.

<img src="media/ACI_diagram_2.jpg" width="500" style="float: left;" alt="Web service deployment workflow">

### Pre-requisites <a id="pre-reqs"></a>
For this notebook to run properly on our machine, an Azure workspace is required. If we don't have one, we need to first run through the short [20_azure_workspace_setup.ipynb](20_azure_workspace_setup.ipynb) notebook to create it.

### Library import <a id="libraries"></a>
Throughout this notebook, we will be using a variety of libraries. We are listing them here for better readibility.

In [1]:
# For automatic reloading of modified libraries
%reload_ext autoreload
%autoreload 2

# Regular python libraries
import os
import sys

# fast.ai
from fastai.vision import models

# Azure
import azureml.core
from azureml.core import Experiment, Workspace
from azureml.core.image import ContainerImage
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException

# Computer Vision repository
sys.path.extend([".", "../.."])
# This "sys.path.extend()" statement allows us to move up the directory hierarchy 
# and access the utils_cv package
from utils_cv.common.deployment import generate_yaml
from utils_cv.common.data import root_path 
from utils_cv.classification.model import IMAGENET_IM_SIZE, model_to_learner

# Check core SDK version number
print(f"Azure ML SDK Version: {azureml.core.VERSION}")

Azure ML SDK Version: 1.0.85


## 2. Model retrieval and export <a id="model"></a>

For demonstration purposes, we will use here a ResNet18 model, pretrained on ImageNet. The following steps would be the same if we had trained a model locally (cf. [**01_training_introduction.ipynb**](01_training_introduction.ipynb) notebook for details).

Let's first retrieve the model.

In [2]:
learn = model_to_learner(models.resnet18(pretrained=True), IMAGENET_IM_SIZE)

To be able to use this model, we need to export it to our local machine. We store it in an `outputs/` subfolder.

In [3]:
output_folder = os.path.join(os.getcwd(), 'outputs')
model_name = 'im_classif_resnet18'  # Name we will give our model both locally and on Azure
pickled_model_name = f'{model_name}.pkl'
os.makedirs(output_folder, exist_ok=True)

learn.export(os.path.join(output_folder, pickled_model_name))

To create or access an Azure ML Workspace, you will need the following information. If you are coming from previous notebook you can retreive existing workspace, or create a new one if you are just starting with this notebook.

- subscription ID: the ID of the Azure subscription we are using
- resource group: the name of the resource group in which our workspace resides
- workspace region: the geographical area in which our workspace resides (e.g. "eastus2" -- other examples are ---available here -- note the lack of spaces)
- workspace name: the name of the workspace we want to create or retrieve.


In [4]:
subscription_id = "YOUR_SUBSCRIPTION_ID"
resource_group = "YOUR_RESOURCE_GROUP_NAME"  
workspace_name = "YOUR_WORKSPACE_NAME"  
workspace_region = "YOUR_WORKSPACE_REGION" #Possible values eastus, eastus2 and so on.

## 3. Model deployment on Azure <a id="deploy"></a>

### 3.A Workspace retrieval <a id="workspace"></a>

In [prior notebook](20_azure_workspace_setup.ipynb) notebook, we created a workspace. This is a critical object from which we will build all the pieces we need to deploy our model as a web service. Let's start by retrieving it.

In [ ]:
# A util method that creates a workspace or retrieves one if it exists, also takes care of Azure Authentication
from utils_cv.common.azureml import get_or_create_workspace

ws = get_or_create_workspace(
        subscription_id,
        resource_group,
        workspace_name,
        workspace_region)

# Print the workspace attributes
print('Workspace name: ' + ws.name, 
      'Workspace region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

### 3.B Model registration <a id="register"></a>

Our final goal is to deploy our model as a web service. To do so, we need to first register it in our workspace, i.e. place it in our workspace's model registry. We can do this in 2 ways:
1. register the model directly
2. upload the model on Azure and then register it there.

The advantage of the first method is that it does not require the setup of an experiment or of any runs. The advantage of the second fashion is that we can keep track of the models that we used or trained in a given experiment, and understand where the ones we ended up registering come from.

The cells below show each of the methods.

#### Without experiment <a id="noexp"></a>

We leverage the `register` method from the Azure ML `Model` object. For that, we just need the location of the model we saved on our local machine, its name and our workspace object.

In [6]:
model = Model.register(
    model_path = os.path.join('outputs', pickled_model_name),
    model_name = model_name,
    tags = {"Model": "Pretrained ResNet18"},
    description = "Image classifier",
    workspace = ws
)

Registering model im_classif_resnet18


#### With an experiment <a id="exp"></a>

An experiment contains a series of trials called `Runs`. A run typically contains some tasks, such as training a model, etc. Through a run's methods, we can log several metrics such as training and test loss and accuracy, and even tag our run. The full description of the run class is available [here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run.run?view=azure-ml-py). In our case, however, we just need the run to attach our model file to our workspace and experiment.

We do this by using `run.upload_file()` and `run.register_model()`, which takes:
- a `model_name` that represents what our model does
- and the `model_path` relative to the run.

Using `run.upload_file()` and specifying the `outputs/` folder allows us to check the presence of the uploaded model on the Azure portal. This is especially convenient when we want to try different versions of a model, or even different models entirely, and keep track of them all, even if we end up registering only the best performing one.

Let's first create a new experiment. If an experiment with the same name already exists in our workspace, the run we will generate will be recorded under that already existing experiment.

In [ ]:
# Create a new/Retrieve an existing experiment
experiment_name = 'image-classifier-webservice'
experiment = Experiment(workspace=ws, name=experiment_name)
print(f"New/Existing experiment:\n \
      --> Name: {experiment.name}\n \
      --> Workspace name: {experiment.workspace.name}")

In [8]:
# Initialize the run
run = experiment.start_logging(snapshot_directory=None)
# "snapshot_directory=None" prevents a snapshot from being saved -- this helps keep the amount of storage used low

Now that we have launched our run, we can see our experiment on the Azure portal, under `Experiments` (in the left-hand side list).

<img src="media/experiment.jpg" width="800" alt="Azure portal view of experiment">

We can now attach our local model to our workspace and experiment.

In [9]:
# Upload the model (.pkl) file to Azure
run.upload_file(
    name=os.path.join('outputs', pickled_model_name), 
    path_or_stream=os.path.join(os.getcwd(), 'outputs', pickled_model_name)
)

In [10]:
# Register the model with the workspace
model = run.register_model(
    model_name=model_name,
    model_path=os.path.join('outputs', pickled_model_name),
    tags = {"Model": "Pretrained ResNet18"},
)
# !!! We need to make sure that the model name we use here is the same as in the scoring script below !!!

Now that the model is uploaded and registered, we can see it on the Azure platform, under `Outputs` and `Models`

<div class="inline-block">
    <img src="media/uploaded_model.jpg" width="800" alt="Azure portal view of the Outputs/ folder">
</div>

<div class="inline-block">
    <img src="media/models.jpg" width="800" alt="Azure portal view of the Models section">
</div>

We can also check that it is programatically accessible

In [11]:
print(f"Model:\n --> Name: {model.name}\n \
      --> ID: {model.id}\n \
      --> Path:{model._get_model_path_remote(model.name, model.version, ws)}")

Model:
 --> Name: im_classif_resnet18
       --> ID: im_classif_resnet18:11
       --> Path:azureml-models\im_classif_resnet18\11\im_classif_resnet18.pkl


In [12]:
run.get_file_names()

['outputs/im_classif_resnet18.pkl']

If we are also interested in verifying which model we uploaded, we can download it to our local machine

In [13]:
model.download(exist_ok=True)

'im_classif_resnet18.pkl'

<i><b>Note:</b> If we ran the cells in both the "with an experiment" and "without experiment" sections, we got 2 iterations of the same model registered on Azure. This is not a problem as any operation that we perform on the "model" object, later on, will be associated with the latest version of the model that we registered. To clean things up, we can go to the portal, select the model we do not want and click the "Delete" button. In general, we would register the model using only one of these 2 methods. </i>

We are all done with our model registration, so we can close our run.

In [14]:
# Close the run
run.complete()

In [ ]:
# Access the portal
run

### 3.C Scoring script <a id="scoring"></a>
For the web service to return predictions on a given input image, we need to provide it with instructions on how to use the model we just registered. These instructions are stored in the scoring script.

This script must contain two required functions, `init()` and `run(input_data)`:
- In the `init()` function, we typically load the model into a global object. This function is executed only once when the Docker container is started.
- In the `run(input_data)` function, the model is used to predict a value based on the input data. The input and output of `run` typically use JSON as serialization and de-serialization format but we are not limited to that.

<i><b>Note:</b> The "run()" function here is different from the "run" object we created in our experiment</i>

This file must also be stored in the current directory.

In [16]:
scoring_script = "score.py"

In [17]:
%%writefile $scoring_script
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import os
import json

from base64 import b64decode
from io import BytesIO

from azureml.core.model import Model
from fastai.vision import load_learner, open_image

def init():
    global model
    model_path = Model.get_model_path(model_name='im_classif_resnet18')
    # ! We cannot use the *model_name* variable here otherwise the execution on Azure will fail !

    model_dir_path, model_filename = os.path.split(model_path)
    model = load_learner(model_dir_path, model_filename)


def run(raw_data):

    # Expects raw_data to be a list within a json file
    result = []    
    
    for im_string in json.loads(raw_data)['data']:
        im_bytes = b64decode(im_string)
        try:
            im = open_image(BytesIO(im_bytes))
            pred_class, pred_idx, outputs = model.predict(im)
            result.append({"label": str(pred_class), "probability": str(outputs[pred_idx].item())})
        except Exception as e:
            result.append({"label": str(e), "probability": ''})
    return result

Overwriting score.py


### 3.D Using `azureml.core.environment` to build the docker image and for deployment <a id="env"></a>

Azure Machine Learning provides an easy way to manage environments through [azureml.core.environment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment?view=azure-ml-py) in the AzureML Python SDK.

`Environments` provide a way to manage software dependencies so that controlled environments are reproducible with minimal manual configuration as you move between local and distributed cloud development environments.

Most importantly, it allows users to encapsulate the various software settings and dependencies in a reproducible manner through the use of containerization and package management tools such as conda, a popular open-source, cross-platform, language-agnostic package management tool.

In order to make predictions on the Azure platform, it is important to create an environment as similar as possible to the one in which the model was trained. Here, we use a fast.ai pretrained model that also requires pytorch and a few other libraries. To re-create this environment, we use a [Docker container](https://www.docker.com/resources/what-container). We configure it via a yaml file that will contain all the conda dependencies needed by the model. This yaml file is a subset of  `<repo_root>/classification/environment.yml`.

In [18]:
# Create a deployment-specific yaml file from classification/environment.yml
try:
    generate_yaml(
        directory=os.path.join(root_path()), 
        ref_filename='environment.yml',
        needed_libraries=['pytorch', 'spacy', 'fastai', 'dataclasses'],
        conda_filename='myenv.yml'
    )
    # Note: Take a look at the generate_yaml() function for details on how to create your yaml file from scratch

except FileNotFoundError:
    raise FileNotFoundError("The *environment.yml* file is missing - Please make sure to retrieve it from the github repository")

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- pytorch==1.2.0
- fastai==1.0.57
- spacy
- dataclasses
channels:
- conda-forge
- defaults
- pytorch
- fastai



We will now use the generated conda dependencies yaml file named `myenv.yml` to create an AzureML `Environment` object from it using the `azureml.core.environment` class.

Then, we will specify a base dockerfile to use to build the docker image that we will be using for deployment, which adds specific packages that are required for inferencing.

Finally, we will register the generated `Environment` object to our workspace with AzureML Environment Management Service (EMS) so that we can retrieve it for reuse throughout Azure Machine Learning Services.

We will be able to retrieve the environment by using the name that we specified (`im_classif_resnet18`) and the workspace that we created. Then we can use the same environment to deploy to other compute targets such as AKS and Azure App Services.

To learn more about how to retrieve and reuse environments in AzureML services, please refer to the [documentation page](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments)

In [19]:
from azureml.core import Environment
from azureml.core.environment import DEFAULT_CPU_IMAGE

cv_test_env = Environment.from_conda_specification(name="im_classif_resnet18", file_path="myenv.yml")

# specifying the latest required inferencing stack to be used for deployment
cv_test_env.inferencing_stack_version="latest"

# We will be using the default CPU image for Azure Machine Learning as the base image
# and will add required packages for inferencing
cv_test_env.docker.base_dockerfile="""FROM {}
RUN apt-get update && \
    apt-get install -y libssl-dev build-essential libgl1-mesa-glx
""".format(DEFAULT_CPU_IMAGE)

# setting docker.base_image to None to use the base_dockerfile specified above to build the image
cv_test_env.docker.base_image=None

# Now, let's try registering the environment. 
# You'll be able to see the specified environment printed out.
cv_test_env.register(ws)

{
    "name": "im_classif_resnet18",
    "version": "1",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge",
                "defaults",
                "pytorch",
                "fastai"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults"
                    ]
                },
                "pytorch==1.2.0",
                "fastai==1.0.57",
                "spacy",
                "dataclasses"
            ],
            "name": "azureml_53f7459b2d11a780edbefaa1f46263fa"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": null,
        "baseDockerfile": "F

In [20]:
# Since building the docker image for the first time requires a few minutes, let's start building the image
# that we'll be using for deployment now and monitor the build through the streamed logs.
cv_test_env.build(ws).wait_for_completion(show_output=True)

Image Build Status: Running

2020/04/09 20:31:52 Downloading source code...
2020/04/09 20:31:53 Finished downloading source code
2020/04/09 20:31:54 Creating Docker network: acb_default_network, driver: 'bridge'
2020/04/09 20:31:54 Successfully set up Docker network: acb_default_network
2020/04/09 20:31:54 Setting up Docker configuration...
2020/04/09 20:31:55 Successfully set up Docker configuration
2020/04/09 20:31:55 Logging in to registry: cvwsbbd59dea.azurecr.io
2020/04/09 20:31:56 Successfully logged into cvwsbbd59dea.azurecr.io
2020/04/09 20:31:56 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/04/09 20:31:56 Scanning for dependencies...
2020/04/09 20:31:57 Successfully scanned dependencies
2020/04/09 20:31:57 Launching container with name: acb_step_0
Sending build context to Docker daemon  61.44kB

Step 1/22 : FROM mcr.microsoft.com/azureml/o16n-base/python-assets:latest AS inferencing-assets
latest: Pulling from azu

Preparing to unpack .../libxdamage1_1%3a1.1.4-2_amd64.deb ...
Unpacking libxdamage1:amd64 (1:1.1.4-2) ...
Selecting previously unselected package libxfixes3:amd64.
Preparing to unpack .../libxfixes3_1%3a5.0.1-2_amd64.deb ...
Unpacking libxfixes3:amd64 (1:5.0.1-2) ...
Selecting previously unselected package libxshmfence1:amd64.
Preparing to unpack .../libxshmfence1_1.2-1_amd64.deb ...
Unpacking libxshmfence1:amd64 (1.2-1) ...
Selecting previously unselected package libxxf86vm1:amd64.
Preparing to unpack .../libxxf86vm1_1%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86vm1:amd64 (1:1.1.4-1) ...
Selecting previously unselected package libtxc-dxtn-s2tc0:amd64.
Preparing to unpack .../libtxc-dxtn-s2tc0_0~git20131104-1.1_amd64.deb ...
Unpacking libtxc-dxtn-s2tc0:amd64 (0~git20131104-1.1) ...
Preparing to unpack .../zlib1g_1%3a1.2.8.dfsg-2ubuntu4.3_amd64.deb ...
Unpacking zlib1g:amd64 (1:1.2.8.dfsg-2ubuntu4.3) over (1:1.2.8.dfsg-2ubuntu4.1) ...
Processing triggers for libc-bin (2.23-0ubuntu11) ...


Setting up software-properties-common (0.96.20.9) ...
Processing triggers for dbus (1.10.6-1ubuntu3.4) ...
gpg: keyring `/tmp/tmpvhm3l443/secring.gpg' created
gpg: keyring `/tmp/tmpvhm3l443/pubring.gpg' created
gpg: requesting key 5234BF2B from hkp server keyserver.ubuntu.com
gpg: /tmp/tmpvhm3l443/trustdb.gpg: trustdb created
gpg: key 5234BF2B: public key "Launchpad PPA for Adiscon" imported
gpg: Total number processed: 1
gpg:               imported: 1  (RSA: 1)
OK
Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
  dh-python distro-info-data file gir1.2-glib-2.0 iso-codes libapt-inst2.0
  libdbus-glib-1-2 libgirepository-1.0-1 libmagic1 libmpdec2 libpython3-stdlib
  libpython3.5-minimal libpython3.5-stdlib lsb-release mime-support
  powermgmt-base python-apt-common python3 python3-apt python3-dbus python3-gi
  python3-minimal python3-pycurl python3-software-properties python


preshed-3.0.2        | 122 KB    |            |   0% 
preshed-3.0.2        | 122 KB    | ########## | 100% 

cymem-2.0.3          | 41 KB     |            |   0% 
cymem-2.0.3          | 41 KB     | ########## | 100% 

jsonschema-3.2.0     | 89 KB     |            |   0% 
jsonschema-3.2.0     | 89 KB     | ########## | 100% 

fastai-1.0.57        | 173 KB    |            |   0% 
fastai-1.0.57        | 173 KB    | 6          |   7% 
fastai-1.0.57        | 173 KB    | ########## | 100% 

cudatoolkit-10.0.130 | 380.0 MB  |            |   0% 
cudatoolkit-10.0.130 | 380.0 MB  |            |   0% 
cudatoolkit-10.0.130 | 380.0 MB  | 1          |   2% 
cudatoolkit-10.0.130 | 380.0 MB  | 3          |   3% 
cudatoolkit-10.0.130 | 380.0 MB  | 5          |   5% 
cudatoolkit-10.0.130 | 380.0 MB  | 6          |   7% 
cudatoolkit-10.0.130 | 380.0 MB  | 8          |   9% 
cudatoolkit-10.0.130 | 380.0 MB  | #          |  11% 
cudatoolkit-10.0.130 | 380.0 MB  | #2         |  13% 
cudatoolkit-10.0.130 | 


cudatoolkit-10.0.130 | 380.0 MB  | ########4  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########4  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########4  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########4  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########4  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########4  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  85% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  86% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  86% 
cudatoolkit-10.0.130 | 380.0 MB  | ########5  |  86% 
cudatoolkit-10.0.130 | 380.

...


cudnn-7.6.5          | 226.4 MB  | #########  |  90% 
cudnn-7.6.5          | 226.4 MB  | #########  |  90% 
cudnn-7.6.5          | 226.4 MB  | #########  |  91% 
cudnn-7.6.5          | 226.4 MB  | #########  |  91% 
cudnn-7.6.5          | 226.4 MB  | #########  |  91% 
cudnn-7.6.5          | 226.4 MB  | #########  |  91% 
cudnn-7.6.5          | 226.4 MB  | #########  |  91% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  91% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  91% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  91% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  91% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  91% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  91% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  92% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  92% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  92% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  92% 
cudnn-7.6.5          | 226.4 MB  | #########1 |  92% 
cudnn-7.6.5          | 226.


mkl-2019.5           | 205.2 MB  | ########4  |  84% 
mkl-2019.5           | 205.2 MB  | ########4  |  84% 
mkl-2019.5           | 205.2 MB  | ########4  |  84% 
mkl-2019.5           | 205.2 MB  | ########4  |  84% 
mkl-2019.5           | 205.2 MB  | ########4  |  84% 
mkl-2019.5           | 205.2 MB  | ########4  |  85% 
mkl-2019.5           | 205.2 MB  | ########4  |  85% 
mkl-2019.5           | 205.2 MB  | ########4  |  85% 
mkl-2019.5           | 205.2 MB  | ########4  |  85% 
mkl-2019.5           | 205.2 MB  | ########4  |  85% 
mkl-2019.5           | 205.2 MB  | ########4  |  85% 
mkl-2019.5           | 205.2 MB  | ########4  |  85% 
mkl-2019.5           | 205.2 MB  | ########4  |  85% 
mkl-2019.5           | 205.2 MB  | ########5  |  85% 
mkl-2019.5           | 205.2 MB  | ########5  |  85% 
mkl-2019.5           | 205.2 MB  | ########5  |  85% 
mkl-2019.5           | 205.2 MB  | ########5  |  85% 
mkl-2019.5           | 205.2 MB  | ########5  |  85% 
mkl-2019.5           | 205.


mkl-2019.5           | 205.2 MB  | #########6 |  97% 
mkl-2019.5           | 205.2 MB  | #########6 |  97% 
mkl-2019.5           | 205.2 MB  | #########6 |  97% 
mkl-2019.5           | 205.2 MB  | #########6 |  97% 
mkl-2019.5           | 205.2 MB  | #########6 |  97% 
mkl-2019.5           | 205.2 MB  | #########6 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  97% 
mkl-2019.5           | 205.2 MB  | #########7 |  98% 
mkl-2019.5           | 205.2 MB  | #########7 |  98% 
mkl-2019.5           | 205.2 MB  | #########7 |  98% 
mkl-2019.5           | 205.2 MB  | #########7 |  98% 
mkl-2019.5           | 205.

  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3923 sha256=696d823b1c81fef17df683ba5c3441ab5db0416bd7be2230899bc478d094315b
  Stored in directory: /root/.cache/pip/wheels/e2/1d/52/535a274b9c2ce7d4064838f2bdb62013801281ef7d7f21e2ee
  Created wheel for liac-arff: filename=liac_arff-2.4.0-py3-none-any.whl size=13333 sha256=2e920f4163d3fe21d07dc9f6b161f3ffb32f6ff0bbc5c139dee1d8dd201d7a9f
  Stored in directory: /root/.cache/pip/wheels/ba/2a/e1/6f7be2e2ea150e2486bff64fd6f0670f4f35f4c8f31c819fb8
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78530 sha256=b2f132b2a0f3ebbe6de68e33e730d17a6bbc2d515c31fd0b16b0823a282b9246
  Stored in directory: /root/.cache/pip/wheels/09/84/74/d2b4feb9ac9488bc83c475cb2cbe8e8b7d9cea8320d32f3787
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10503 sha256=e9607443d8dfeaae6c454bb9c0ef6a88d97288cc5bfd349494cd971288a6b3e7
  Stored in directory: /root/.cache/pip/wheels/21/5c/83/1dd7

7049b31c7d29: Pushed
1bf21c5cbd6d: Pushed
dcf2f1c2d5fe: Pushed
a49a0ffb7d87: Pushed
a52b426fb3ad: Pushed
7ca872f79cb3: Pushed
e1171d4d60ca: Layer already exists
85389f9ead9e: Layer already exists
6ef1a8ae63b7: Layer already exists
f2608f66a0e3: Layer already exists
0e259b09e5f4: Layer already exists
340dc32eb998: Layer already exists
df18b66efaa6: Layer already exists
ccdb13a20bf2: Layer already exists
9513cdf4e497: Layer already exists
29f36b5893dc: Layer already exists
7f083f9454c0: Layer already exists
597926253e5a: Pushed
5351bc49e1c0: Pushed
b156ccc9c968: Pushed
latest: digest: sha256:f9f43c62c1a99f949d80c18af70fdd175cebc8a0994cc0d3a8f505612ca3a01b size: 4727
2020/04/09 20:47:40 Successfully pushed image: cvwsbbd59dea.azurecr.io/azureml/azureml_a9f62c4b6fbf408218d5b60ae4f43f11:latest
2020/04/09 20:47:40 Step ID: acb_step_0 marked as successful (elapsed time in seconds: 698.968631)
2020/04/09 20:47:40 Populating digests for step ID: acb_step_0...
2020/04/09 20:47:43 Successfully po

### 3.E Setting up compute resources - Azure Container Instance (ACI) <a id="compute"></a>
We will now use the docker image that was built from the environment that we specified to deploy the model to Azure Container Instance (ACI).

In this notebook, we use [Azure Container Instances](https://docs.microsoft.com/en-us/azure/container-instances/container-instances-overview) (ACI) which are good for quick and [cost-effective](https://azure.microsoft.com/en-us/pricing/details/container-instances/) development/test deployment scenarios.

To set them up properly, we need to indicate the number of CPU cores and the amount of memory we want to allocate to our web service. Optional tags and descriptions are also available for us to identify the instances in AzureML when looking at the `Compute` tab in the Azure Portal.

<b>NOTE:</b> Since the docker image was built in the previous step, we will be able to reuse the built image to deploy the model to other compute targets as well, allowing us to reduce the time spent on building the docker images.

To learn more about reusing environments across Azure Machine Learning, please refer to the [documentation page](
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments)

<i><b>Note:</b> For production workloads, it is better to use [Azure Kubernetes Service](https://docs.microsoft.com/en-us/azure/aks/) (AKS) instead. We will demonstrate how to do this in the [next notebook](22_deployment_on_azure_kubernetes_service.ipynb).<i>

In [21]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'im-classif-websvc'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='score.py', environment=cv_test_env)

# If you would like to increase the number of cpu cores and memory used for the webservice,
# update the corresponding settings for cpu_cores and memory_gb
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

### 3.F Web service deployment <a id="websvc"></a>

The final step to deploying our web service is to call `WebService.deploy_from_image()`. This function uses the Docker image and the deployment configuration we created above to perform the following:

- Deploy the docker image to an Azure Container Instance
- Call the `init()` function in our scoring file
- Provide an HTTP endpoint for scoring calls

In [22]:
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
# Deploy the web service
service.wait_for_deployment(show_output=True)

Running................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


When successful, we expect to see the following:

`
Creating service
Running .....
SucceededACI service creation operation finished, operation "Succeeded"`

In the case where the deployment is not successful, we can look at the image and service logs to debug. [These instructions](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-troubleshoot-deployment) can also be helpful.

In [ ]:
# Access the service logs
# print(service.get_logs())

In [23]:
# Retrieve the service status
print(f"Service {service.name} is _{service.state}_ and available at {service.scoring_uri}")

Service im-classif-websvc is _Healthy_ and available at http://49381d74-9c78-4f1c-b784-ae8baab16766.eastus.azurecontainer.io/score


We can also check the presence and status of both our new Docker image and web service on the Azure portal, under the `Images` and `Deployments` tabs, respectively.


<img src="media/docker_images.jpg" width="800" alt="Azure portal view of the Images section">
<img src="media/deployments.jpg" width="800" alt="Azure portal view of the Deployments section">

## 4. Notes on web service deployment <a id="notes"></a>

As we discussed above, Azure Container Instances tend to be used to develop and test deployments. They are typically configured with CPUs, which usually suffice when the number of requests per second is not too high. When working with several instances, we can configure them further by specifically [allocating CPU resources](https://docs.microsoft.com/en-us/azure/container-instances/container-instances-container-groups#deployment) to each of them.

For production requirements, i.e. when &gt; 100 requests per second are expected, we recommend deploying models to Azure Kubernetes Service (AKS). It is a convenient infrastructure as it manages hosted Kubernetes environments, and makes it easy to deploy and manage containerized applications without container orchestration expertise. It also supports deployments with CPU clusters and deployments with GPU clusters.

We will see an example of this in the [next notebook](22_deployment_on_azure_kubernetes_service.ipynb).

## 5. Clean up <a id="clean"></a>

Throughout the notebook, we used a workspace and Azure container instances. To get a sense of the cost we incurred, we can refer to this [calculator](https://azure.microsoft.com/en-us/pricing/calculator/). We can also navigate to the [Cost Management + Billing](https://ms.portal.azure.com/#blade/Microsoft_Azure_Billing/ModernBillingMenuBlade/Overview) pane on the portal, click on our subscription ID, and click on the Cost Analysis tab to check our credit usage.

In order not to incur extra costs, let's delete the resources we no longer need.

Once we have verified that our web service works well on ACI (cf. "Next steps" section below), we can delete it. This helps reduce [costs](https://azure.microsoft.com/en-us/pricing/details/container-instances/), since the container group we were paying for no longer exists, and allows us to keep our workspace clean.

In [ ]:
# service.delete()

If our goal is to continue using our workspace, we should keep it available. On the contrary, if we plan on no longer using it and its associated resources, we can delete it.

<i><b>Note:</b> Deleting the workspace will delete all the experiments, outputs, models, Docker images, deployments, etc. that we created in that workspace</i>

In [ ]:
# ws.delete(delete_dependent_resources=True)
# This deletes our workspace, the container registry, the account storage, Application Insights and the key vault

## 6. Next steps <a id="next-steps"></a>

In the [next tutorial](22_deployment_on_azure_kubernetes_service.ipynb), we will leverage the same Docker image, and deploy our model on AKS. We will then test both of our web services in the [23_aci_aks_web_service_testing.ipynb](23_aci_aks_web_service_testing.ipynb) notebook.